In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

In [2]:
img_height = 100
img_width = 100
batch_size = 32


In [3]:
# Define train and test directories
train_dir = 'dataset/train'
test_dir = 'dataset/test'

In [4]:
# Use ImageDataGenerator for data augmentation and preprocessing
train_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

# Create data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 60298 images belonging to 8 classes.
Found 2344 images belonging to 8 classes.


In [5]:
from tensorflow.keras.metrics import categorical_accuracy, top_k_categorical_accuracy
def accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=2)

In [6]:
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization, Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.optimizers import Adam

model = Sequential()

# First Conv layer with updated input_shape
model.add(Conv2D(64, (2, 2), input_shape=(100, 100, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

# Second Conv layer
model.add(Conv2D(512, (2, 2), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Dropout(0.3))

# Third Conv layer
model.add(Conv2D(1024, (2, 2), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Dropout(0.3))

# Fourth Conv layer
model.add(Conv2D(1024, (1, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(1, 1)))
model.add(BatchNormalization())

model.add(Dropout(0.3))

# Fifth Conv layer
model.add(Conv2D(1024, (1, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(1, 1)))
model.add(BatchNormalization())

model.add(Dropout(0.3))

model.add(Flatten())

# Fully connected layers
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

# Output layer with 7 classes
model.add(Dense(8, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 99, 99, 64)        832       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 49, 49, 64)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 49, 49, 64)        256       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 512)       131584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 512)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 24, 24, 512)       2048      
_________________________________________________________________
dropout (Dropout)            (None, 24, 24, 512)       0

In [ ]:
22,066,088
36,066,120

In [7]:
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau
early=EarlyStopping(monitor='accuracy',patience=4,mode='auto')
reduce_lr = ReduceLROnPlateau(monitor='accuracy', factor=0.5, patience=2, verbose=1,cooldown=0, mode='auto',min_delta=0.0001, min_lr=0)

In [8]:
# Compute class weights
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
class_labels = list(train_generator.class_indices.keys())
class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights = dict(enumerate(class_weights))
class_weights

{0: 1.176408615576713,
 1: 0.9401584133715853,
 2: 1.094893957001743,
 3: 0.941215034965035,
 4: 0.9374689054726368,
 5: 0.9613839285714286,
 6: 1.0226933514246948,
 7: 0.9748124676668392}

In [9]:
class_labels

['actinic keratosis',
 'basal cell carcinoma',
 'dermatofibroma',
 'melanoma',
 'nevus',
 'pigmented benign keratosis',
 'squamous cell carcinoma',
 'vascular lesion']

In [10]:
class_weights = {0: 1,
 1: 1,
 2: 1,
 3: 1,
 4: 0.5,
 5: 1,
 6: 1,
 7: 1}

In [11]:
model.fit(train_generator,
          epochs=50,
          batch_size=batch_size,
          class_weight=class_weights,
          validation_data=test_generator,
          callbacks=[early,reduce_lr])

Epoch 1/50
1885/1885 [==============================] - 247s 128ms/step - loss: 2.7261 - accuracy: 0.2160 - val_loss: 2.2745 - val_accuracy: 0.1195
Epoch 2/50
1885/1885 [==============================] - 134s 71ms/step - loss: 1.6730 - accuracy: 0.2716 - val_loss: 2.0217 - val_accuracy: 0.2018
Epoch 3/50
1885/1885 [==============================] - 138s 73ms/step - loss: 1.6413 - accuracy: 0.2880 - val_loss: 1.9960 - val_accuracy: 0.1152
Epoch 4/50
1885/1885 [==============================] - 140s 74ms/step - loss: 1.6154 - accuracy: 0.2976 - val_loss: 2.0629 - val_accuracy: 0.2402
Epoch 5/50
1885/1885 [==============================] - 193s 102ms/step - loss: 1.5877 - accuracy: 0.3145 - val_loss: 1.9998 - val_accuracy: 0.2223
Epoch 6/50
1885/1885 [==============================] - 265s 141ms/step - loss: 1.5368 - accuracy: 0.3333 - val_loss: 2.5977 - val_accuracy: 0.1288
Epoch 7/50
1885/1885 [==============================] - 556s 295ms/step - loss: 1.5224 - accuracy: 0.3443 - val_los

KeyboardInterrupt: 

In [14]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True, mode='min')
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=3, verbose=1,
    cooldown=1, mode='auto', min_delta=0.0001, min_lr=0.00001
)


In [15]:
history = model.fit(train_generator,
                    initial_epoch=25,  # Start from the 18th epoch (0-indexed)
                    epochs=50,
                    batch_size=batch_size,
                    class_weight=class_weights,
                    validation_data=test_generator,
                    callbacks=[early,reduce_lr, checkpoint])

Epoch 26/50
1885/1885 [==============================] - 129s 68ms/step - loss: 0.7339 - accuracy: 0.7151 - val_loss: 1.8057 - val_accuracy: 0.7159
Epoch 27/50
1885/1885 [==============================] - 131s 70ms/step - loss: 0.7049 - accuracy: 0.7242 - val_loss: 1.7347 - val_accuracy: 0.3963
Epoch 28/50
1885/1885 [==============================] - 135s 72ms/step - loss: 0.6821 - accuracy: 0.7326 - val_loss: 1.0849 - val_accuracy: 0.7210
Epoch 29/50
1885/1885 [==============================] - 133s 71ms/step - loss: 0.6683 - accuracy: 0.7383 - val_loss: 1.3097 - val_accuracy: 0.6259
Epoch 30/50
1885/1885 [==============================] - 134s 71ms/step - loss: 0.6433 - accuracy: 0.7475 - val_loss: 1.7383 - val_accuracy: 0.4863
Epoch 31/50
 187/1885 [=>............................] - ETA: 1:59 - loss: 0.6144 - accuracy: 0.7612

KeyboardInterrupt: 

In [16]:
best_epoch = history.history['val_loss'].index(min(history.history['val_loss'])) + 1
print(f"Best epoch was {best_epoch}")

NameError: name 'history' is not defined

In [17]:
import numpy as np

# Load model
model = tf.keras.models.load_model('best_model.h5')

# Load validation data
validation_generator = test_datagen.flow_from_directory(
    'dataset/test',
    target_size=(100, 100),
    batch_size=batch_size,
    class_mode='categorical'
)


# Get precision, recall, and F1 score for each class
from sklearn.metrics import classification_report

y_pred = model.predict(validation_generator)
y_pred = np.argmax(y_pred, axis=1)
y_true = validation_generator.classes
class_labels = list(validation_generator.class_indices.keys())

print(classification_report(y_true, y_pred, target_names=class_labels))

Found 2344 images belonging to 8 classes.
                            precision    recall  f1-score   support

         actinic keratosis       0.00      0.00      0.00        30
      basal cell carcinoma       0.07      0.08      0.08       124
            dermatofibroma       0.00      0.00      0.00        32
                  melanoma       0.12      0.13      0.13       261
                     nevus       0.67      0.65      0.66      1547
pigmented benign keratosis       0.12      0.13      0.13       268
   squamous cell carcinoma       0.00      0.00      0.00        46
           vascular lesion       0.00      0.00      0.00        36

                  accuracy                           0.46      2344
                 macro avg       0.12      0.12      0.12      2344
              weighted avg       0.47      0.46      0.47      2344



## Inception V3

In [18]:
from tensorflow.keras.applications import InceptionV3
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(100, 100, 3))

# Unfreeze all layers for fine-tuning
for layer in base_model.layers:
    layer.trainable = True

# Add custom classification layers on top
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
predictions = tf.keras.layers.Dense(8, activation='softmax')(x)

# Create the model
model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100, 100, 3) 0                                            
__________________________________________________________________________________________________
conv2d_5 (Conv2D)               (None, 49, 49, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_5 (BatchNor (None, 49, 49, 32)   96          conv2d_5[0][0]                   
__________________________________________________________________________________________________
activation (Activation)         (None, 49, 49, 32)   0           batch_normalization_5[0][0]      
______________________________________________________________________________________________

In [ ]:
['actinic keratosis',
 'basal cell carcinoma',#
 'dermatofibroma',
 'melanoma', #
 'nevus',
 'pigmented benign keratosis',
 'squamous cell carcinoma',#
 'vascular lesion']

In [19]:
class_weights = {0: 1.176408615576713,
 1: 0.9,
 2: 1,
 3: 0.9,
 4: 0.3,
 5: 0.9,
 6: 1,
 7: 1}

In [20]:
checkpoint = ModelCheckpoint('best_model_inception.h5', monitor='val_loss', save_best_only=True, mode='min')


In [21]:
history = model.fit(train_generator,
                    # initial_epoch=25,  # Start from the 18th epoch (0-indexed)
                    epochs=50,
                    batch_size=batch_size,
                    class_weight=class_weights,
                    validation_data=test_generator,
                    callbacks=[early,reduce_lr, checkpoint])

Epoch 1/50
1885/1885 [==============================] - 83s 41ms/step - loss: 1.3609 - accuracy: 0.3928 - val_loss: 1.7476 - val_accuracy: 0.3191
Epoch 2/50
1885/1885 [==============================] - 73s 39ms/step - loss: 1.2201 - accuracy: 0.4692 - val_loss: 1.8882 - val_accuracy: 0.2521
Epoch 3/50
1885/1885 [==============================] - 75s 40ms/step - loss: 1.2155 - accuracy: 0.4605 - val_loss: 1.9344 - val_accuracy: 0.2564
Epoch 4/50
1885/1885 [==============================] - 73s 39ms/step - loss: 1.0836 - accuracy: 0.5205 - val_loss: 1.6652 - val_accuracy: 0.3660
Epoch 5/50
1885/1885 [==============================] - 72s 38ms/step - loss: 1.0481 - accuracy: 0.5342 - val_loss: 1.8993 - val_accuracy: 0.3366
Epoch 6/50
1885/1885 [==============================] - 74s 39ms/step - loss: 0.9574 - accuracy: 0.5808 - val_loss: 1.3629 - val_accuracy: 0.5299
Epoch 7/50
1885/1885 [==============================] - 80s 42ms/step - loss: 1.0518 - accuracy: 0.5392 - val_loss: 1.7632 -

KeyboardInterrupt: 

In [22]:
import numpy as np

# Load model
model = tf.keras.models.load_model('best_model_inception.h5')

# Load validation data
validation_generator = test_datagen.flow_from_directory(
    'dataset/test',
    target_size=(100, 100),
    batch_size=batch_size,
    class_mode='categorical'
)


# Get precision, recall, and F1 score for each class
from sklearn.metrics import classification_report

y_pred = model.predict(validation_generator)
y_pred = np.argmax(y_pred, axis=1)
y_true = validation_generator.classes
class_labels = list(validation_generator.class_indices.keys())

print(classification_report(y_true, y_pred, target_names=class_labels))

Found 2344 images belonging to 8 classes.
                            precision    recall  f1-score   support

         actinic keratosis       0.02      0.03      0.03        30
      basal cell carcinoma       0.05      0.07      0.06       124
            dermatofibroma       0.01      0.03      0.02        32
                  melanoma       0.10      0.14      0.12       261
                     nevus       0.65      0.51      0.58      1547
pigmented benign keratosis       0.12      0.17      0.14       268
   squamous cell carcinoma       0.03      0.04      0.04        46
           vascular lesion       0.03      0.03      0.03        36

                  accuracy                           0.38      2344
                 macro avg       0.13      0.13      0.13      2344
              weighted avg       0.46      0.38      0.41      2344



In [23]:
checkpoint = ModelCheckpoint('best_model_inception_v2.h5', monitor='val_loss', save_best_only=True, mode='min')


In [26]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Load and prepare the data with data augmentation
train_generator = train_datagen.flow_from_directory(
    'dataset/train_1',
    target_size=(100, 100),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    'dataset/test_1',
    target_size=(100, 100),
    batch_size=batch_size,
    class_mode='categorical'
)

from tensorflow.keras import backend as K

def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        cross_entropy = -y_true * K.log(y_pred)
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy
        return K.sum(loss, axis=1)
    return focal_loss_fixed

# Example usage with model compilation
model.compile(optimizer='adam',
              loss=focal_loss(gamma=2., alpha=0.25),
              metrics=['accuracy'])

history = model.fit(train_generator,
                    # initial_epoch=25,  # Start from the 18th epoch (0-indexed)
                    epochs=50,
                    batch_size=batch_size,
                    class_weight=class_weights,
                    validation_data=test_generator,
                    callbacks=[early,reduce_lr, checkpoint])

Found 9376 images belonging to 8 classes.
Found 2344 images belonging to 8 classes.
Epoch 1/50
293/293 [==============================] - 33s 98ms/step - loss: 0.0843 - accuracy: 0.6471 - val_loss: 0.1090 - val_accuracy: 0.6805
Epoch 2/50
293/293 [==============================] - 25s 84ms/step - loss: 0.0759 - accuracy: 0.6694 - val_loss: 0.1289 - val_accuracy: 0.6578
Epoch 3/50
293/293 [==============================] - 25s 84ms/step - loss: 0.0701 - accuracy: 0.6767 - val_loss: 0.1273 - val_accuracy: 0.6335
Epoch 4/50
293/293 [==============================] - 25s 85ms/step - loss: 0.0676 - accuracy: 0.6767 - val_loss: 0.1310 - val_accuracy: 0.6284

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 5/50
293/293 [==============================] - 25s 84ms/step - loss: 0.0632 - accuracy: 0.6929 - val_loss: 0.1148 - val_accuracy: 0.6881
Epoch 6/50
293/293 [==============================] - 24s 83ms/step - loss: 0.0614 - accuracy: 0.7138 - val_loss: 0

In [30]:
import numpy as np
from sklearn.metrics import classification_report

# Load the custom loss function
custom_loss = focal_loss(gamma=2., alpha=0.25)

# Load the model using the custom_objects parameter
model = tf.keras.models.load_model('best_model_inception_v2.h5', custom_objects={'focal_loss_fixed': custom_loss})

# Re-compile the model with the custom loss function
model.compile(optimizer='adam',
              loss=custom_loss,
              metrics=['accuracy'])

# Load validation data
validation_generator = test_datagen.flow_from_directory(
    'dataset/test',
    target_size=(100, 100),
    batch_size=batch_size,
    class_mode='categorical'
)

# Get precision, recall, and F1 score for each class
y_pred = model.predict(validation_generator)
y_pred = np.argmax(y_pred, axis=1)
y_true = validation_generator.classes
class_labels = list(validation_generator.class_indices.keys())

print(classification_report(y_true, y_pred, target_names=class_labels))


Found 2344 images belonging to 8 classes.
                            precision    recall  f1-score   support

         actinic keratosis       0.00      0.00      0.00        30
      basal cell carcinoma       0.12      0.11      0.12       124
            dermatofibroma       0.00      0.00      0.00        32
                  melanoma       0.11      0.15      0.13       261
                     nevus       0.66      0.59      0.62      1547
pigmented benign keratosis       0.12      0.17      0.14       268
   squamous cell carcinoma       0.00      0.00      0.00        46
           vascular lesion       0.05      0.06      0.05        36

                  accuracy                           0.43      2344
                 macro avg       0.13      0.13      0.13      2344
              weighted avg       0.47      0.43      0.45      2344

